# Calculate Forward Transfer (FWT)

This notebook is for calculating Forward Transfer (FWT) metric of continual learning experiments. We support calculating for multiple experiments (with respect to one reference independent learning experiment) in an output folder at the same time.

Please specify all the settings (the cells tagged "Please Specify") following the instructions and Hit "Run All". 

## 1. Specify Output Folder

Please specify the output folder path that you want to calcualate FWT. 

In [95]:
group_folder_path: str = "../outputs/til_pmnist_finetuning/"

Please specify the accuracy CSV file of the reference independent learning experiment. 

In [ ]:
ilref_acc_path: str = "../outputs/til_pmnist_independent/acc.csv"

Please specify the relative path (with respect to result folder) of accuracy CSV file. Default is `results/acc.csv`.

In [96]:
acc_relative_path: str = "results/acc.csv"

Please specify the relative path (with respect to result folder) of calculated FWT CSV file. Default is `results/fwt.csv`.

In [ ]:
fwt_relative_path: str = "results/fwt.csv"

Building paths...

In [ ]:
import os

output_folder_names = [f for f in os.listdir(group_folder_path) if os.path.isdir(os.path.join(group_folder_path, f))]

## 2. Calculate FWT

We read the accuracy CSV files into Pandas Dataframes and calculate FWT, then save to CSV files.

In [ ]:
import pandas as pd


for folder_name in output_folder_names:
    folder_path = os.path.join(group_folder_path, folder_name)
    acc_path = os.path.join(folder_path, acc_relative_path)
    fwt_path = os.path.join(folder_path, fwt_relative_path)

    if not os.path.exists(acc_path):
        print(f"File {acc_path} does not exist. Skipping...")
        continue
    
    acc_df = pd.read_csv(acc_path)
    ilref_acc_df = pd.read_csv(ilref_acc_path)
    
    # Get the number of tasks
    num_tasks = acc_df.shape[0]
    
    fwt_data = []
    for N in range(2, num_tasks + 1):  # skip the first task. FWT cannot be defined for it
        fwt_N = 0.0
        for t in range(2, N + 1):
            a_t_I = ilref_acc_df.loc[ilref_acc_df['after_training_task'] == t, f'test_on_task_{t}'].values
            a_t_t = acc_df.loc[acc_df['after_training_task'] == t, f'test_on_task_{t}'].values
            fwt_N += (a_t_t - a_t_I)
        fwt_N /= (N - 1)
        fwt_data.append({'after_training_task': N, 'FWT': fwt_N})
    
    fwt_df = pd.DataFrame(fwt_data)

    # Save FWT to a CSV file
    fwt_df.to_csv(fwt_path)
    print(f"Saved FWT to {fwt_path}")